In [ ]:
# adaptation of CAN_response.m by Carl Tape
# initial coding by: Yongki Aiman, Tomy Gunawan, Angel Ling
# further coding by Amanda McPherson, 2020

# the example waveform is from CAN (Canberra, Australia) for the 2004 Mw 9.X Sumatra-Andaman earthquake
# custom functions: markp_minutes, wf_fft

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pylab as plt
import warnings

from matplotlib import dates
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

from lib_seis import markp_minutes, wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# getting the waveform

client = Client("IRIS")

# 2004 Sumatra-Andaman earthquake
origintime = UTCDateTime("2004-12-26T00:58:53")
starttime  = origintime - (     12 * 60 * 60 )           # half a day before origin time
endtime    = origintime + ( 9 * 24 * 60 * 60)            # 9 days after origin time

st = client.get_waveforms("G", "CAN", "*", "LHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting the seismogram

can = st.copy()                           

t   = can[0].times()                                           # time axis for plotting in seconds
tr  = can[0].data                                              

title = f'{can[0].stats["station"]} ({can[0].stats["channel"]}) − starting {can[0].stats["starttime"]}'

# plot the seismogram with time in seconds
plt.figure()
plt.plot(t, tr, 'b') 
plt.title(title)
#plt.ylim([-3E4, 3E4])
plt.xlabel('time [s]')
plt.ylabel('counts');

In [ ]:
# label the x-axis using absolute time

can_dates = st.copy().resample(0.1)

date = can_dates[0].times(type='utcdatetime')                      # time axis for plotting absolute time
date = dates.date2num(date)

# plot the seismogram with respect to absolute time
plt.figure()
plt.plot_date(date, can_dates[0].data, 'b')
plt.title(title)
plt.xlabel('dates')
plt.ylabel('counts')
ax = plt.gca()
ax.xaxis.set_tick_params(rotation=30, labelsize=10)

In [ ]:
# spectral analysis

st1 = st.copy()
st1.detrend('constant')
st1.detrend('linear')
st1.taper(max_percentage=0.2, type="cosine")

tr = st1[0].data

sampling_frequency = st1[0].stats.sampling_rate
Nyquist_frequency  = sampling_frequency / 2

fft_amp, fft_phase, fft_freq = wf_fft(tr, Nyquist_frequency)

C = fft_amp * np.exp( 1j * fft_phase )                     # Fourier transform reconstructed from its amplitude and phase

In [ ]:
# template code for using markp_minutes to identify the periods (in minutes) of spectral peaks

#fig=plt.figure();
#YOUR PLOT COMMAND HERE
#cid = fig.canvas.mpl_connect('button_press_event', markp_minutes) #displays the value 1/x/60 for a selected point (x,y)